# Introduction

Hello from my new project. In this project, I tried to detect masked people and unmasked people using the yolov7 library. Steps of this project, Firstly I downloaded the dataset from Kaggle, and I will share with you the link of the dataset in this chapter. Then I visualized images of the dataset and reviewed XML format folders. Images tagged in xml format are not compatible for yolov7 so I converted xml files to txt format and saved the txt files in a file. And then I created a new dataset for yolov7 using text file and image files. I split the txt folder and the image folder into train, test and Val folders, as yolov7 wanted and I specified ways of these folders to yolov7 with "data.yaml". After I created to new dataset, I downloaded the yolov7 library and installed it. After I installed the yolov7 library I made to train with new dataset and I visualized the results. After the training, I detected masked and unmasked people in a picture I downloaded. I am currently on the way of improving myself. In this learning path, I would be very happy if you would examine my project and help me with how to improve my project or what I should do in this development path. I wish you a good day and good work.

[For a detailed description of the dataset, please click here](https://www.kaggle.com/datasets/andrewmvd/face-mask-detection)

Firstly, I made this project at the Google colab. If you want to look at the colab version, [click here.](https://colab.research.google.com/drive/1fcZqEgRgAE8neo-TG2QSdCuo0setQUvS?usp=sharing)

# Content <a id = "cont"></a>
- [Data Rewiew](#1)
- [Preparing DataSet](#2)
    - [Translate ".xml" format to ".txt" format](#2a)
    - [Create Dataset For Yolov7](#2b)
- [Yolov7](#3)
    - [Download Yolov7](#3a)
    - [Traning](#3b)
    - [Result Visualization](#3c)
    - [Detection](#3d)
- [CONCLUSION](#4)

# Data Rewiew <a id = "1"></a>
[Go Content](#cont)

In this chapter, I reviewed the data in the dataset.

In [ ]:
# I import libraries that I will use.
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

# For preparing
import xml.etree.cElementTree as ET
import glob
import os
import json
import random
import shutil

from PIL import Image, ImageOps

XML file of the image labeled in XML format.

In [ ]:
with open('/content/face-mask-detection/annotations/maksssksksss0.xml') as f:
    contents = f.read()
    print(contents)

I visualized a few images in the dataset.

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss0.png")

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss1.png")

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss100.png")

In [ ]:
# creating a image1 object
im1 = Image.open(r"/content/face-mask-detection/images/maksssksksss100.png")
 
# applying grayscale method
im2 = ImageOps.grayscale(im1)

im2

# Preparing DataSet <a id = "2"></a>
[Go Content](#cont)

In this chapter, I prepared the dataset for the yolov7.

## Translate ".xml" format to ".txt" format <a id = "2a"></a>
[Go Content](#cont)

Firstly, I translated xml format to txt format. While I made this work, I used this article If you want to check, [click here.](https://towardsdatascience.com/convert-pascal-voc-xml-to-yolo-for-object-detection-f969811ccba5)

In [ ]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    
    return [x_center, y_center, width, height]

def yolo_to_xml_bbox(bbox, w, h):
    # x_center, y_center, width, height
    w_half_len = (bbox[2] * w) / 2
    h_half_len = (bbox[3] * h) / 2
    
    xmin = int((bbox[0] * w) - w_half_len)
    ymin = int((bbox[1] * h) - h_half_len)
    xmax = int((bbox[0] * w) + w_half_len)
    ymax = int((bbox[1] * h) + h_half_len)
    
    return [xmin, ymin, xmax, ymax]

classes = []

input_dir = "/content/face-mask-detection/annotations"
output_dir = "/content/labels"
image_dir = "/content/face-mask-detection/images"

os.mkdir(output_dir)

if not os.path.isdir(output_dir):
    os.mkdir(output_dir)

import glob

files = glob.glob(os.path.join(input_dir, "*.xml"))
for fil in files:
    basename = os.path.basename(fil)
    filename = os.path.splitext(basename)[0]
    if not os.path.exists(os.path.join(image_dir, f"{filename}.png")):
        print(f"{filename} image does not exist!")
        continue
    
    result = []
    
    # Parse the content of the xml file
    tree = ET.parse(fil)
    root = tree.getroot()
    width = int(root.find("size").find("width").text)
    height = int(root.find("size").find("height").text)
    
    for obj in root.findall("object"):
        label = obj.find("name").text
        
        # check for new classes and append to list
        if label not in classes:
            classes.append(label)
        index = classes.index(label)
        pil_bbox = [int(x.text) for x in obj.find("bndbox")]
        yolo_bbox = xml_to_yolo_bbox(pil_bbox, width, height)
        
        # convert data to string
        bbox_string = " ".join([str(x) for x in yolo_bbox])
        result.append(f"{index} {bbox_string}")
    
    if result:
        # generate a yolo format text file for each xml file
        with open(os.path.join(output_dir, f"{filename}.txt"), "w", encoding = "utf-8") as f:
            f.write("\n".join(result))
# generate the classes file as reference
with open("/content/classes.txt", "w", encoding = "utf-8") as f:
    f.write(json.dumps(classes))

In the below code, I specified the classes of the dataset.

In [ ]:
with open('/content/classes.txt') as f:
    contents = f.read()
    print(contents)

In [ ]:
Image.open("/content/face-mask-detection/images/maksssksksss0.png")

xml format of the maksssksksss0 image

In [ ]:
with open('/content/face-mask-detection/annotations/maksssksksss0.xml') as f:
    contents = f.read()
    print(contents)

txt format of the maksssksksss0 image

In [ ]:
with open('/content/labels/maksssksksss0.txt') as f:
    contents = f.read()
    print(contents)

## Create Dataset For Yolov7 <a id = "2b"></a>
[Go Content](#cont)

In this chapter, I created a dataset for yolov7.

In [ ]:
os.mkdir("/content/data/")
os.mkdir('/content/data/train')
os.mkdir('/content/data/val')
os.mkdir('/content/data/test')
os.mkdir('/content/data/train/images')
os.mkdir('/content/data/train/labels')
os.mkdir('/content/data/test/images')
os.mkdir('/content/data/test/labels')
os.mkdir('/content/data/val/images')
os.mkdir('/content/data/val/labels')

In the below code, I added image names of in the dataset in the list. Because I used it when created the dataset. 

In [ ]:
metarial = []

for i in os.listdir("/content/face-mask-detection/images"):
    srt = i[:-4]
    metarial.append(srt)

In [ ]:
len(metarial)

In [ ]:
# a few image names of in the dataset
metarial[0:10]

In the below code, Created function. Actually, I split the dataset into train, test and val.

In [ ]:
def preparinbdata(main_txt_file, main_img_file, train_size, test_size, val_size):
    
    for i in range(0,train_size):
        
        source_txt = main_txt_file + "/" + metarial[i] + ".txt"
        source_img = main_img_file + "/" + metarial[i] + ".png"
        
        mstring = metarial[i]
        train_destination_txt = "/content/data/train/labels" + "/" + metarial[i] + ".txt" 
        train_destination_png = "/content/data/train/images" + "/" + metarial[i] + ".png" 
                
        shutil.copy(source_txt, train_destination_txt)
        shutil.copy(source_img, train_destination_png)
                
        #metarial.remove(file_name[:-4])
                
                
    for l in range(train_size , train_size + test_size):
        
        source_txt = main_txt_file + "/" + metarial[l] + ".txt"
        source_img = main_img_file + "/" + metarial[l] + ".png"
        
        mstring = metarial[l]
        test_destination_txt = "/content/data/test/labels" + "/" + metarial[l] + ".txt"
        test_destination_png = "/content/data/test/images" + "/" + metarial[l] + ".png"
                
        shutil.copy(source_txt, test_destination_txt)
        shutil.copy(source_img, test_destination_png)
                
        #metarial.remove(file_name[:-4])
                
                
    for n in range(train_size + test_size , train_size + test_size + val_size):
        
        source_txt = main_txt_file + "/" + metarial[n] + ".txt"
        source_img = main_img_file + "/" + metarial[n] + ".png"
        
        mstring = metarial[n]
        val_destination_txt = "/content/data/val/labels" + "/" + metarial[n] + ".txt"
        val_destination_png = "/content/data/val/images" + "/" + metarial[n] + ".png"
                
        shutil.copy(source_txt, val_destination_txt)
        shutil.copy(source_img, val_destination_png)
                
        #metarial.remove(file_name[:-4])

In [ ]:
preparinbdata("/content/labels", "/content/face-mask-detection/images", 603, 150, 100)

I created ".yaml" folder.

In [ ]:
# configure .yaml file to guide the model for training
%cd /content/data

yaml_text = """train: /content/data/train/images
val: /content/data/val/images

nc: 3
names: ["without_mask", "with_mask", "mask_weared_incorrect"]"""

with open("/content/data/data.yaml", 'w') as file:
    file.write(yaml_text)

%cat data/data.yaml

# Yolov7 <a id = "3"></a>
[Go Content](#cont)

In this chapter, I used the yolov7.

## Download Yolov7 <a id = "3a"></a>
[Go Content](#cont)

In this chapter, I downloaded the yolov7 library and the pre-trained ".pt" file.

In [ ]:
!# Download YOLOv7 code
%cd /content/
!git clone https://github.com/WongKinYiu/yolov7
%cd yolov7
!ls

In [ ]:
!# Download trained weights
!wget https://github.com/WongKinYiu/yolov7/releases/download/v0.1/yolov7.pt

## Traning <a id = "3b"></a>
[Go Content](#cont)

In this chapter, I made to train with yolov7 and the dataset.

In [ ]:
%cd /content/

In [ ]:
# Train
!python /content/yolov7/train.py --workers 8 --device 0 --batch-size 16 --epochs 50 --data /content/data/data.yaml  --cfg /content/yolov7/cfg/training/yolov7.yaml --weights '' --name yolov7_1 --hyp /content/yolov7/data/hyp.scratch.p5.yaml

## Result Visualization <a id = "3c"></a>
[Go Content](#cont)

In this chapter, I visualized the training result of the yolov7.

In [ ]:
Image.open("/content/runs/train/yolov7_1/results.png")

In [ ]:
Image.open("/content/runs/train/yolov7_1/train_batch0.jpg")

In [ ]:
Image.open("/content/runs/train/yolov7_1/train_batch9.jpg")

In the code below, I detected masked and unmasked people in the pictures in the test dataset.

In [ ]:
!# Detection
!python /content/yolov7/detect.py --weights /content/runs/train/yolov7_1/weights/best.pt --conf 0.25 --img-size 640 --source /content/data/test/images/

I visualized results of test detected.

In [ ]:
Image.open("/content/runs/detect/exp/maksssksksss1.png")

In [ ]:
Image.open("/content/runs/detect/exp/maksssksksss508.png")

## Detection <a id = "3d"></a>
[Go Content](#cont)

In this part, I detected the masked and unmasked people in a random photo I downloaded from the internet with the yolov7 library.

In [ ]:
# I downloaded one image for use at the detect.
%cd /content
!wget "https://onecms-res.cloudinary.com/image/upload/s--XV7DHKzY--/c_fill,g_auto,h_468,w_830/f_auto,q_auto/people-wearing-mask-at-orchard-road-singapore-feb-3--49-.jpg?itok=GdDk1T6A"

In [ ]:
!# Detection
!python /content/yolov7/detect.py --weights /content/runs/train/yolov7_1/weights/best.pt --conf 0.25 --img-size 640 --source /content/image0.jpg

In [ ]:
Image.open("/content/runs/detect/exp3/image0.jpg")

# CONCLUSION <a id = "4"></a>
[Go Content](#cont)

In this my work, I worked on yolov7 library. In the future, I will continue to make computer vision projects similar to this project. I am learning the English language newly if I make an errata please advise me in the comment. Thank you for reading my notebook, your votes and your comments. I will be waiting for your advice.